In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip -q install google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown, HTML # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  text = text.replace('$', r'\$')

  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Passo para autenticar no gmail

# PRÉ-REQUISITOS:
# 1- Ative a Gmail API: https://console.cloud.google.com/apis/library/gmail.googleapis.com
# 2- Crie um OAuth 2.0 Client ID no Google Cloud Console
#     .Tipo: “Área de trabalho” (Desktop app)
#     .Faça o download do JSON com as credenciais (credentials.json) - será solicitado upload nesta célula

import os
import base64
import json
from email.message import EmailMessage
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.colab import files

# Para verificar a versão da biblioteca de forma mais robusta
try:
    from importlib.metadata import version
    google_auth_oauthlib_version = version('google-auth-oauthlib')
except ImportError:
    # Fallback para Python < 3.8 (menos comum em Colab recente)
    try:
        import pkg_resources
        google_auth_oauthlib_version = pkg_resources.get_distribution('google-auth-oauthlib').version
    except Exception:
        google_auth_oauthlib_version = "Não foi possível determinar (tente !pip show google-auth-oauthlib)"


# Escopos
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def gmail_authenticate():
    creds = None
    creds_file_path = 'credentials.json'

    print(f"--- Verificando a versão da biblioteca google-auth-oauthlib ---")
    print(f"Versão detectada: {google_auth_oauthlib_version}")
    print("--- Fim da verificação da versão ---")


    if not os.path.exists(creds_file_path):
        print(f"ERRO: O arquivo {creds_file_path} NÃO FOI ENCONTRADO. Faça o upload novamente.")
        print("Por favor, faça o upload do seu arquivo credentials.json:")
        uploaded = files.upload()
        if creds_file_path not in uploaded:
            print(f"ERRO: Upload do {creds_file_path} falhou ou nome incorreto.")
            return None
        print(f"Arquivo {creds_file_path} carregado.")

    try:
        with open(creds_file_path, 'r') as f:
            client_config_loaded = json.load(f)
            if 'installed' in client_config_loaded and 'redirect_uris' in client_config_loaded['installed']:
                print(f"URIs de redirecionamento encontradas no JSON: {client_config_loaded['installed']['redirect_uris']}")
            else:
                print("AVISO: Estrutura esperada ('installed' com 'redirect_uris') não encontrada no JSON.")
    except Exception as e:
        print(f"Erro ao ler ou processar o {creds_file_path}: {e}")
        return None

    if os.path.exists('token.json'):
        try:
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
            print("Token.json encontrado e carregado.")
        except Exception as e:
            print(f"Erro ao carregar token.json: {e}. Removendo token inválido.")
            if os.path.exists('token.json'):
                 os.remove('token.json')
            creds = None

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("Token expirado, tentando refresh...")
            try:
                creds.refresh(Request())
                print("Token atualizado com sucesso.")
            except Exception as e:
                print(f"Falha ao atualizar token: {e}")
                creds = None

        if not creds or not creds.valid:
            print("Iniciando novo fluxo de autorização...")
            try:
                flow = InstalledAppFlow.from_client_secrets_file(creds_file_path, SCOPES)

                print(f"Flow redirect_uri (após init): {flow.redirect_uri if hasattr(flow, 'redirect_uri') else 'N/A'}")

                # ****** A CORREÇÃO PRINCIPAL ESTÁ AQUI ******
                if flow.redirect_uri is None:
                    print("ATENÇÃO: flow.redirect_uri está None. Configurando manualmente para OOB.")
                    flow.redirect_uri = "urn:ietf:wg:oauth:2.0:oob"
                print(f"Flow redirect_uri (após possível correção manual): {flow.redirect_uri}")
                # ******************************************

                auth_url, _ = flow.authorization_url(prompt='consent')

                print(f"--- URL de Autorização Gerada ---")
                print(auth_url)
                print("--- Fim da URL de Autorização ---")
                print(f"\n🔗 Acesse este link para autorizar: {auth_url}")

                code = input("Cole aqui o código de autorização: ")
                flow.fetch_token(code=code)
                creds = flow.credentials
                print("Token obtido com sucesso.")
            except Exception as e:
                print(f"ERRO CRÍTICO durante o fluxo de autenticação: {e}")
                import traceback
                traceback.print_exc()
                return None

        with open('token.json', 'w') as token:
            token.write(creds.to_json())
            print("Token salvo em token.json.")

    try:
        service = build('gmail', 'v1', credentials=creds)
        print("Serviço Gmail construído com sucesso.")
        return service
    except Exception as e:
        print(f"Erro ao construir o serviço Gmail: {e}")
        import traceback
        traceback.print_exc()
        return None

# Lembre-se de excluir token.json e credentials.json se quiser forçar o fluxo completo
# if os.path.exists('token.json'):
#     os.remove('token.json')
#     print("token.json removido para novo teste.")
# if os.path.exists('credentials.json'):
#     os.remove('credentials.json')
#     print("credentials.json removido para novo teste.")

print("Iniciando o processo de autenticação...")
gmail_service = gmail_authenticate()
if gmail_service:
    print("Autenticação e construção do serviço Gmail bem-sucedidas!")
else:
    print("Falha na autenticação ou construção do serviço Gmail.")

In [ ]:
### Agente Buscador de Notícias ###

def buscador_noticias (topico, data_de_hoje):


    buscador_noticias = Agent(
        name="buscador_noticias",
        model="gemini-2.0-flash",
        description="""Agente que busca notícias para o clipping""",
        tools=[google_search],
        instruction="""
        Você é um redator de clipping de notícias.
        Sua função é pesquisar as última notícias relevantes dos últimos dias sobre o tópico
        (topico) especificado.
        Você pesquisa estas notícias em inglês, espanhol e português, utilizado o motor de busca
        do Google (google_search). Os tópicos estarão separados pelo símbolo ";" e você deve realizar
        buscas por cada tópico separadamente.
        Você trará os resultados de maneira organizada e consolidará notícias sobre o mesmo assunto
        em um único texto resumido fazendo referência para os seus respectivos links (os nomeie pelo
        domínio de onde encontrou, exemplo "New York Times" entre parêntesis e com um hiperlink
        clicável para a notícia - é muito importante ter o hiperlink clicável).
        Você revisará em sua memória se você já trouxe este tópico anteriormente. Caso você já tenha
        mencionado este tópico você o ignorará e o substituirá por outro tópico relevante, se houver.
        Traga as 5 notícias que considerar mais relevantes rankeadas por ordem de relevância.
        Você definirá esta relevância com base no engajamento e número de links que mencionam a notícia.
        MUITO IMPORTANTE: Os textos resultantes serão em HTML, então use sua escrita de acordo,
        por exemplo:
        - Use <b></b> para negrito, ao invés de ** **. Exemplo <b> Palavra em Negrito </b>
        - **Importante para HTML:** Cada parágrafo deve ser envolvido pela tag `<p></p>`. Pular linha com "enter" não funciona em HTML para criar novos parágrafos. Certifique-se de que todo bloco de texto que seria um parágrafo normal esteja entre `<p>` e `</p>`.

        Além disso sempre comece o texto com o formato: "As 5 notícias mais relevantes dos últimos dias sobre [tópico] são:"

        Em resumo, o formato de entrega deve ser:

        "As 5 notícias mais relevantes dos últimos dias sobre [tópico] são:

        <p>1. <b>Título Tema 1:</b> Descrição tema 1 (Fonte tema 1, <a href="URL_DA_NOTICIA_1">clique aqui para ler</a>).</p>
        <p>2. <b>Título Tema 2:</b> Descrição tema 2 (Fonte tema 2, <a href="URL_DA_NOTICIA_2">clique aqui para ler</a>).</p>
        <p>3. <b>Título Tema 3:</b> Descrição tema 3 (Fonte tema 3, <a href="URL_DA_NOTICIA_3">clique aqui para ler</a>).</p>
        <p>4. <b>Título Tema 4:</b> Descrição tema 4 (Fonte tema 4, <a href="URL_DA_NOTICIA_4">clique aqui para ler</a>).</p>
        <p>5. <b>Título Tema 5:</b> Descrição tema 5 (Fonte tema 5, <a href="URL_DA_NOTICIA_5">clique aqui para ler</a>).</p>

        <p>Atenciosamente,</p>

        <p><b>Agente de Notícias</b></p>"

        NUNCA ESQUEÇA DOS HYPERLINKS CLICÁVEIS!

        """

        )

    entrada_buscador_noticias = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    noticias = call_agent(buscador_noticias, entrada_buscador_noticias)

    return noticias



In [ ]:
### Agente Enviador de Email ###

def send_email(to, subject, message_text):
    service = gmail_service # Usa a variável global que já foi autenticada

    if not service:
        print("ERRO: Serviço Gmail não está autenticado. Não é possível enviar e-mail.")
        return

    message = EmailMessage()
    message.set_content(message_text, subtype='html')
    message['To'] = to
    message['From'] = "me" # A API do Gmail enviará como o usuário autenticado
    message['Subject'] = subject

    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
    create_message = {
        'raw': encoded_message
    }

    try:
        send_message = service.users().messages().send(userId="me", body=create_message).execute()
        print(f"✅ Mensagem enviada! ID: {send_message['id']}")
    except Exception as e:
        print(f"❌ Erro ao enviar e-mail: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
### Orquestrador ###

data_de_hoje = date.today().strftime("%d/%m/%Y")
print("🚀 Iniciando o Sistema de Criação de Clipping de Notícias🚀")

# --- Obter o Tópico do Usuário ---
topico = input("Por favor, digite o TÓPICO sobre o qual você quer criar o clipping: ")
email_destinatario = input("Para qual e-mail você deseja enviar este clipping?  ")

    # Validação simples de e-mail (pode ser melhorada)
if "@" not in email_destinatario or "." not in email_destinatario:
        print("⚠️ E-mail do destinatário parece inválido. Por favor, verifique.")

# Inserir lógica do sistema de agentes ################################################

noticias = buscador_noticias(topico, data_de_hoje)

print("Resultados buscados")
display(HTML(noticias))
print("------------")

print("Conteúdo HTML gerado pelo agente:")
print(noticias)
print("------------")

# envio = send_email(to, subject, noticias)

assunto_do_email = f"Clipping de Notícias: '{topico}' - {data_de_hoje}"

print(f"\n📤 Enviando e-mail para {email_destinatario} com o assunto: {assunto_do_email}...")
send_email(to=email_destinatario,
               subject=assunto_do_email,
               message_text=noticias)


🚀 Iniciando o Sistema de Criação de Clipping de Notícias🚀
Por favor, digite o TÓPICO sobre o qual você quer criar o clipping: Portuguesa
Para qual e-mail você deseja enviar este clipping?  gabrielschiavoni@gmail.com
Resultados buscados


------------
Conteúdo HTML gerado pelo agente:
As 5 notícias mais relevantes dos últimos dias sobre Portuguesa são:

<p>1. <b>Vitória da Portuguesa sobre o Porto Vitória-ES:</b> A Portuguesa venceu o Porto Vitória-ES por 3 a 1, isolando-se na liderança da Série D. O NETLUSA acompanhou o jogo ao vivo e trouxe informações sobre a escalação da Lusa (NETLUSA, <a href="https://www.netlusa.com.br/">clique aqui para ler</a>).</p>

<p>2. <b>Portuguesa goleia o Rio Claro no Paulista Sub-20:</b> A Portuguesa Sub-20 segue com 100% de aproveitamento no Campeonato Paulista após golear o Rio Claro (NETLUSA, <a href="https://www.netlusa.com.br/">clique aqui para ler</a>).</p>

<p>3. <b>Técnico da Portuguesa na mira de clube da Série B:</b> Além das notícias sobre os jogos, o NETLUSA informou que o técnico da Portuguesa está sendo sondado por um clube da Série B (NETLUSA, <a href="https://www.netlusa.com.br/">clique aqui para ler</a>).</p>

<p>4. <b>Portuguesa toma decisão sobre transmissão da 2ª roda